In [367]:
library(data.table)
library(ggplot2)
library(latex2exp)


In [368]:
time <- 500

In [369]:
N_AGENTS <- 12625

In [370]:
mypath <- "../../log/ProvaEmailcontacts-act-1"

In [371]:
mypath <- "/home/nik/ProvaEmailcontacts2.5/ProvaEmailcontacts2.5"

In [372]:
list.dirs <- function(path=".", pattern=NULL, all.dirs=FALSE,
  full.names=FALSE, ignore.case=FALSE) {
  # use full.names=TRUE to pass to file.info
  all <- list.files(path, pattern, all.dirs,
           full.names=TRUE, recursive=FALSE, ignore.case)
  dirs <- all[file.info(all)$isdir]
  # determine whether to return full names or just dir names
  if(isTRUE(full.names))
    return(dirs)
  else
    return(basename(dirs))
}

In [373]:
thepath = list.dirs(mypath)[1]

In [374]:
filename = paste(mypath, '/', thepath, '/', 'k_core.csv', sep = "")

In [375]:
DF <- read.csv(file = filename, skip=12)

In [376]:
tempDF <- DF[DF$state =='i',]
tempDF <- tempDF[tempDF$time == 1,]

In [377]:
tempDF

,time,agent,k.core,state,degree
1600,1,1599,2,i,2


In [378]:
agent <- as.integer(tempDF['agent'])
kcore <- as.integer(tempDF['k.core'])

In [379]:
DF$'agent.init' <- agent
DF$'k.core.init' <- kcore
DF$'density' <- 1

In [380]:
DT <- data.table(DF)

In [381]:
DT1 = DT[, sum(density), by=list(time, state)]

In [382]:
names(DT1)[names(DT1)=="V1"] <- "density"

In [383]:
DT1$density <- DT1$density / N_AGENTS

In [384]:
DT1$'agent.init' <- agent
DT1$'k.core.init' <- kcore

In [385]:
tempDT1 <- DT1[DT1$state == 'r']
tempDT1 <- tempDT1[tempDT1$time == 500]
density <- as.numeric(tempDT1[,'density'])

In [386]:
DT1 = DT1[DT1$state == 'r']
DT1[,state:=NULL]

## All datasets

Assemble everything and do the loops

In [ ]:
emptytable <- data.table()
lista = list.dirs(mypath)
for (i in lista){    
    filename = paste(mypath, '/', i, '/', 'k_core.csv', sep = "")
    print(filename)
    DF <- read.csv(file = filename, skip=12)
    tempDF <- DF[DF$state =='i',]
    tempDF <- tempDF[tempDF$time == 1,]

    agent <- as.integer(tempDF['agent'])
    kcore <- as.integer(tempDF['k.core'])

    DF$'agent.init' <- agent
    DF$'k.core.init' <- kcore
    DF$'density' <- 1
    DF$'density.mean' <- 1
    DF$'density.std' <- 1

    DT <- data.table(DF)
    DT1 = DT[, sum(density), by=list(time, state)]
    
    names(DT1)[names(DT1)=="V1"] <- "density"

    DT1$density <- DT1$density / N_AGENTS

    DT1$'agent.init' <- agent
    DT1$'k.core.init' <- kcore
    
    tempDT1 <- DT1[DT1$state == 'r']
    tempDT1 <- tempDT1[tempDT1$time == 500]
    density <- as.numeric(tempDT1[,'density'])
    
    DT1 = DT1[DT1$state == 'r']
    DT1[,state:=NULL]
    #print(DT1)
    emptytable <- rbind(emptytable, DT1)
}

In [ ]:
rawtable <- emptytable

Compute mean and standard deviation and add to data table

In [ ]:
emptytable <- rawtable

In [ ]:
emptytabletime <- emptytable[emptytable$time == 500]
#emptytabletime <- emptytabletime[emptytabletime$density < 0.46]
#emptytabletime <- emptytabletime[emptytabletime$density > 0.05]
#emptytabletime$k.core.init <- emptytabletime$k.core.init + 1 + emptytabletime$k.core.init %% 2 
toplot <- emptytabletime[, mean(density), by=list(time, k.core.init)][,'k.core.init']
toplot$mean <- emptytabletime[emptytabletime$time == 500][, mean(density), by=list(time, k.core.init)][,'V1']
toplot$std <- emptytabletime[emptytabletime$time == 500][, sd(density), by=list(time, k.core.init)][,'V1']

Clean NaN values

In [ ]:
toplot[is.na(toplot)] <- 0

In [ ]:
toplot

Plot

In [ ]:
plot(toplot$k.core.init, toplot$mean,
    ylim=range(c(toplot$mean-toplot$std, toplot$mean+toplot$std)),
     #log='y',
     xlab=TeX('$k_S$'), ylab=TeX('$M_{k_S}$'),
    #main="Scatter plot with std.dev error bars"
)
arrows(toplot$k.core.init, toplot$mean-toplot$std, 
       toplot$k.core.init, toplot$mean+toplot$std, length=0.05, angle=90, code=3)


Log y plot

In [ ]:
plot(toplot$k.core.init, toplot$mean,
     ylim=range(c(toplot$mean-toplot$std, toplot$mean+toplot$std)),
     log='y',
     xlab=TeX('$k_S$'), ylab=TeX('$M_{k_S}$'),
    #main="Scatter plot with std.dev error bars"
)
arrows(toplot$k.core.init, toplot$mean-toplot$std, 
       toplot$k.core.init, toplot$mean+toplot$std, length=0.05, angle=90, code=3)